In [1]:
# This code is part of Qiskit.
#
# (C) Copyright IBM 2017, 2020.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

def required_result_qubits(quadratic, linear, offset):
    
    bounds = []  # bounds = [minimum value, maximum value]
    for condition in [lambda x: x < 0, lambda x: x > 0]:
        bound = 0
        bound += sum(sum(q_ij for q_ij in q_i if condition(q_ij)) for q_i in quadratic)
        bound += sum(l_i for l_i in linear if condition(l_i))
        bound += offset if condition(offset) else 0
        bounds.append(bound)

    # the minimum number of qubits is the number of qubits needed to represent
    # the minimum/maximum value plus one sign qubit
    num_qubits_for_min = int(np.ceil(np.log2(max(-bounds[0], 1))))
    num_qubits_for_max = int(np.ceil(np.log2(bounds[1] + 1)))
    num_result_qubits = 1 + max(num_qubits_for_min, num_qubits_for_max)

    return num_result_qubits

In [ ]:
from qiskit.circuit.library import QFT

linear = []
for var in bqm.variables:
    linear.append(bqm.get_linear(var))
quadratic = bqm.to_numpy_matrix(variable_order=bqm.variables)
offset = bqm.offset

n = np.max([1, len(linear), len(quadratic)])
m = required_result_qubits(quadratic, linear, offset)

print("Input qubits: ", n)
print("Result qubits: ", m)

def apply_state_preparation_A(circuit, n, m, qr_input, qr_result):
    scaling = np.pi * 2 ** (1 - m)
    qr_result = qr_result[:-1]
    circuit.h(qr_result)
    circuit.h(qr_input)

    # constant coefficient
    if offset != 0:
        for i, q_i in enumerate(qr_result):
            circuit.p(scaling * 2**i * offset, q_i)

    # the linear part consists of the vector and the diagonal of the
    # matrix, since x_i * x_i = x_i, as x_i is a binary variable
    for j in range(n):
        value = linear[j] if linear is not None else 0
        value += quadratic[j][j] if quadratic is not None else 0
        if value != 0:
            for i, q_i in enumerate(qr_result):
                circuit.cp(scaling * 2**i * value, qr_input[j], q_i)

    # the quadratic part adds A_ij and A_ji as x_i x_j == x_j x_i
    for j in range(n):
        for k in range(j + 1, n):
            value = quadratic[j][k] + quadratic[k][j]
            if value != 0:
                for i, q_i in enumerate(qr_result):
                    circuit.mcp(scaling * 2**i * value, [qr_input[j], qr_input[k]], q_i)

    # add the inverse QFT
    iqft = QFT(m, do_swaps=False).inverse().reverse_bits()
    circuit.compose(iqft, qubits=qr_result[:], inplace=True)
    
    return circuit

The output circuit is very long again but it is visualized in `kakuro_gas.txt` file.

In [ ]:
#%%capture
#circuit.draw(filename = 'kakuro_gas.txt', fold = 250)

#### Oracle operator $O$

In [ ]:
def get_oracle_O(n, m):
    # Build negative value oracle O.
    
    qr_key_value = QuantumRegister(n + m)
    oracle_bit = QuantumRegister(1, "oracle")
    oracle = QuantumCircuit(qr_key_value, oracle_bit)
    oracle.z(n)  # recognize negative values.

    def is_good_state(measurement):
        """Check whether ``measurement`` is a good state or not."""
        value = measurement[
            n : n + m
        ]
        return value[0] == "1"

    return oracle, is_good_state


#### Grover's algorithm

In [ ]:
def apply_GAS(n, m):
    qr_input = QuantumRegister(n, name = 'var')
    qr_result = QuantumRegister(m + 1, 'res')
    circuit = QuantumCircuit(qr_input, qr_result)
    circuit = apply_state_preparation_A(circuit, n, m, qr_input, qr_result)
    oracle, is_good_state = get_oracle_O(n, m)
    circuit.compose(oracle, inplace=True)
    return circuit
    

In [ ]:
%%capture
circuit = apply_GAS(n, m)
circuit.draw(filename = 'kakuro_gas.txt', fold = 250)

In [ ]:
print('Gate counts: ', circuit.count_ops())
print('Depth: ', circuit.depth())

In [ ]:
from qiskit.utils import algorithm_globals
from qiskit.algorithms import AmplificationProblem, Grover

qr_input = QuantumRegister(n, name = 'var')
qr_result = QuantumRegister(m + 1, 'res')
circuit = QuantumCircuit(qr_input, qr_result)
circuit = apply_state_preparation_A(circuit, n, m, qr_input, qr_result)

oracle, is_good_state = get_oracle_O(n, m)

measurement = False #not self.quantum_instance.is_statevector

rotation_count = algorithm_globals.random.integers(0, 1)

amp_problem = AmplificationProblem(
    oracle=oracle,
    state_preparation=circuit,
    is_good_state=is_good_state,
)

grover = Grover()

circuit = grover.construct_circuit(
    problem=amp_problem, power=rotation_count, measurement=measurement
)

In [ ]:
%%capture
circuit.draw(filename = 'kakuro_gas_full.txt', fold = 250)

In [ ]:
print('Gate counts: ', circuit.count_ops())
print('Depth: ', circuit.depth())

In [ ]:
outcome = self._measure(circuit)
k = int(outcome[0:n_key], 2)
v = outcome[n_key : n_key + n_value]
int_v = self._bin_to_int(v, n_value) + threshold
logger.info("Outcome: %s", outcome)
logger.info("Value Q(x): %s", int_v)

#### Run whole algorithm

In [ ]:
import math

# Variables for tracking the optimum.
optimum_found = False
optimum_key = math.inf
optimum_value = math.inf
threshold = 0
n_key = len(bqm.variables)
n_value = 4

# Variables for tracking the solutions encountered.
num_solutions = 2**n_key
keys_measured = []

# Variables for result object.
operation_count = {}
iteration = 0

# Variables for stopping if we've hit the rotation max.
rotations = 0
max_rotations = int(np.ceil(100 * np.pi / 4))

# Initialize oracle helper object.
qr_key_value = QuantumRegister(n_key + n_value)
#orig_constant = problem_.objective.constant
measurement = not self.quantum_instance.is_statevector
oracle, is_good_state = get_oracle_O(n_key, n_value)


while not optimum_found:
    m = 1
    improvement_found = False

    # Get oracle O and the state preparation operator A for the current threshold.
    problem_.objective.constant = orig_constant - threshold
    a_operator = self._get_a_operator(qr_key_value, problem_)

    # Iterate until we measure a negative.
    loops_with_no_improvement = 0
    while not improvement_found:
        # Determine the number of rotations.
        loops_with_no_improvement += 1
        rotation_count = algorithm_globals.random.integers(0, m)
        rotations += rotation_count
        # Apply Grover's Algorithm to find values below the threshold.
        # TODO: Utilize Grover's incremental feature - requires changes to Grover.
        
        
        
        
        #amp_problem = AmplificationProblem(
        #    oracle=oracle,
        #    state_preparation=a_operator,
        #    is_good_state=is_good_state,
        #)
        #grover = Grover()
        #circuit = grover.construct_circuit(
        #    problem=amp_problem, power=rotation_count, measurement=measurement
        #)